In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/russia-real-estate-2021/input_data.csv


In [4]:
import numpy as np
import pandas as pd 

In [5]:
InputDir="/kaggle/input/russia-real-estate-2021/input_data.csv"
df = pd.read_csv(InputDir,sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 15 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date           object 
 1   price          int64  
 2   level          int64  
 3   levels         int64  
 4   rooms          int64  
 5   area           float64
 6   kitchen_area   float64
 7   geo_lat        float64
 8   geo_lon        float64
 9   building_type  int64  
 10  object_type    int64  
 11  postal_code    float64
 12  street_id      float64
 13  id_region      int64  
 14  house_id       float64
dtypes: float64(7), int64(7), object(1)
memory usage: 1.3+ GB


In [6]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


In [7]:
df.isnull().sum()

date                   0
price                  0
level                  0
levels                 0
rooms                  0
area                   0
kitchen_area           0
geo_lat                0
geo_lon                0
building_type          0
object_type            0
postal_code       507771
street_id        4205554
id_region              0
house_id         3261943
dtype: int64

**删去有空值的列**

In [8]:
df = df.dropna(subset=['house_id','street_id','postal_code'])
df.shape

(6809106, 15)

**删除DataFrame中id_region列中值为200的所有行**

In [9]:
df = df.drop(df[df.id_region==200].index)

**删除重复**

In [10]:
df = df.drop_duplicates()

**清理数据**

In [11]:
df.drop('date', axis=1, inplace=True)

In [12]:
df["object_type"] = df["object_type"].apply(lambda x: 2 if x == 11 else x)

In [13]:
df['rooms'] = df["rooms"].apply(lambda x: 0 if x < 0 else x)

In [14]:
df["price"] = df["price"].abs()

min_price = 800000
max_price = 50000000

df = df[(df["price"] <= max_price) & (df["price"] >= min_price)]

In [15]:
df.loc[df["level"] > df["levels"], "level"] = df["levels"]

In [16]:
min_area = 10
max_area = 150

min_kitchen_area = 6
max_kitchen_area = 40

df = df[(df["area"] <= max_area) & (df["area"] >= min_area)]
df = df[(df["kitchen_area"] <= max_kitchen_area) & (df["kitchen_area"] >= min_kitchen_area)]

In [17]:
df['postal_code'] = df['postal_code'].astype(int)
df['street_id'] = df['street_id'].astype(int)
df['house_id'] = df['house_id'].astype(int)

In [18]:
df.head()

,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
2,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701,242543,50,681306
4,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960,439378,23,1730985
5,1450000,5,5,2,47.0,6.0,48.511172,44.566846,2,0,400096,260588,34,1009994
6,9000000,2,4,3,107.4,21.3,55.009914,82.934859,4,0,630102,233285,54,2823596
8,2300000,16,18,1,39.7,11.5,45.003869,39.086511,4,0,350065,523822,23,1284243


In [19]:
df.to_csv("/kaggle/working/cleaned_realestate.csv",index=False)